In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
import numpy as np

In [6]:
output_notebook()

Loading BokehJS ...

In [10]:
# Function to work with comlex numbers
def tov(cn):
    return np.array([cn.real,cn.imag])
def atob(a,b):
    return np.array([i for i in zip(tov(a),tov(b))])
def mid(a,b):
    return 0.5*(a+b)
def perp(a,b,s=1):
    return mid(a,b) + s*(mid(a,b)-a)*1j
def cmod(a):
    return (a*a.conj()).real**0.5

# Geometry: find an intersection of two lines given four points
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x+y*1j
    
def inter_c(points):
    return intersection(
        line(tov(points[0]),tov(points[1])),
        line(tov(points[2]),tov(points[3]))
    )    

In [48]:
plot = figure(plot_width=400, plot_height=600,match_aspect=True)

circ = np.exp(1j*np.linspace(0,2*np.pi,100))
source = ColumnDataSource(data=dict(x=circ.real, y=circ.imag))
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

alpha = 2
beta = 2.8
gamma = 2
delta = 2.8

c0 = 2+2j

a = np.exp(1j*alpha) + c0
b = np.exp(1j*beta) + c0
z = 1.7*np.exp(1j*gamma) + c0

m1 = mid(a,b)
p1 = perp(a,b,-1)
m2 = mid(b,z)
p2 = perp(b,z,-1)

c = inter_c([m1,p1,m2,p2])
ncirc = circ*cmod(c-a) + c
s1 = ColumnDataSource(data=dict(x=circ.real, y=circ.imag))

plot.line('x','y',source=s1,line_width=3,line_alpha=0.6,color='red')
plot.circle(*tov(c0))
plot.circle(*tov(c),color='red')
plot.circle(0,0)

slider = Slider(start=0, end=10, value=1, step=.1, title="foo")

update_curve = CustomJS(args=dict(source=s1, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = x[i] + f
    }  
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)

show(column(slider,plot))